In [ ]:
import os
import cv2
import tqdm
import mediapy
import numpy as np

from discoverse import DISCOVERSE_ROOT_DIR

video_dir = os.path.join(DISCOVERSE_ROOT_DIR, "data/plate_coffeecup_white")

In [ ]:
arm_videos = []
global_videos = []

listdir = sorted(os.listdir(video_dir))
for i, forder in tqdm.tqdm(enumerate(listdir)):
    if i >= 25:
        break
    # c1_dir = os.path.join(video_dir, forder, "arm_video.mp4")
    # v1 = mediapy.read_video(c1_dir)
    # arm_videos.append(v1)
    c2_dir = os.path.join(video_dir, forder, "global_video.mp4")
    v2 = mediapy.read_video(c2_dir)
    global_videos.append(v2)

In [ ]:
h_or_v = None

# output_size = (2560, 1440)
# output_size = (1440, 1920)
output_size = [1600, 1200]

img_rows = 5
img_cols = 5

ori_img_size = global_videos[0].shape[1:3]

if h_or_v == "h":
    ori_bg_img = np.zeros((img_rows*ori_img_size[0], img_cols*2*ori_img_size[1], 3), dtype=np.uint8)
    output_size[0] *= 2
elif h_or_v == "v":
    ori_bg_img = np.zeros((img_rows*2*ori_img_size[0], img_cols*ori_img_size[1], 3), dtype=np.uint8)
    output_size[1] *= 2
else:
    ori_bg_img = np.zeros((img_rows*ori_img_size[0], img_cols*ori_img_size[1], 3), dtype=np.uint8)

frame_total_min = 1e6

for img in arm_videos:
    frame_total_min = min(frame_total_min, len(img))
for img in global_videos:
    frame_total_min = min(frame_total_min, len(img))

big_video = []
for frame_idx in tqdm.trange(frame_total_min):
    for i in range(img_rows*img_cols):
        if i >= img_rows * img_cols:
            break
        row = i // img_cols
        col = i % img_cols
        if i < len(arm_videos):
            img_1 = arm_videos[i][frame_idx]
        img_2 = global_videos[i][frame_idx]
        if h_or_v == "h":
            ori_bg_img[row*ori_img_size[0]:(row+1)*ori_img_size[0], col*2*ori_img_size[1]:(col*2+1) * ori_img_size[1]] = img_1
            ori_bg_img[row*ori_img_size[0]:(row+1)*ori_img_size[0], (col*2+1)*ori_img_size[1]:(col*2+2) * ori_img_size[1]] = img_2
        elif h_or_v == "v":
            ori_bg_img[row*2*ori_img_size[0]:(row*2+1)*ori_img_size[0], col*ori_img_size[1]:(col+1)*ori_img_size[1]] = img_1
            ori_bg_img[(row*2+1)*ori_img_size[0]:(row*2+2)*ori_img_size[0], col*ori_img_size[1]:(col+1)*ori_img_size[1]] = img_2
        else:
            ori_bg_img[row*ori_img_size[0]:(row+1)*ori_img_size[0], col*ori_img_size[1]:(col+1)*ori_img_size[1]] = img_2
        # cv2.imshow("img", ori_bg_img)
        # key = cv2.waitKey(1)
        # if key == ord("q"):
        #     break
    big_img = cv2.resize(ori_bg_img, output_size, interpolation=cv2.INTER_LINEAR)
    big_video.append(big_img)
mediapy.write_video("big_video.mp4", big_video, fps=30)

In [4]:
bgr_img = cv2.cvtColor(ori_bg_img, cv2.COLOR_RGB2BGR)
cv2.imshow("img", bgr_img)
cv2.waitKey(0)
cv2.destroyAllWindows()